In [1]:
from IPython.display import HTML, Javascript, clear_output
import os
import sys

# Define the repo URL and folder name
REPO_URL = "https://github.com/rosinusserrano/autoencoding_experiments.git"
LOCAL_DIR = "/content/autoencoding_experiments"

# Inject a floating button with JavaScript
def inject_floating_button():
    display(HTML("""
    <script>
    function reloadRepo() {
        google.colab.kernel.invokeFunction('notebook.reload_repo', [], {});
    }
    const button = document.createElement('button');
    button.innerHTML = 'Reload Repo';
    button.style.position = 'fixed';
    button.style.bottom = '20px';
    button.style.right = '20px';
    button.style.backgroundColor = '#ff5050';
    button.style.color = 'white';
    button.style.border = 'none';
    button.style.padding = '10px 20px';
    button.style.borderRadius = '5px';
    button.style.boxShadow = '0px 4px 6px rgba(0,0,0,0.1)';
    button.style.cursor = 'pointer';
    button.onclick = reloadRepo;
    document.body.appendChild(button);
    </script>
    """))

def clone_and_import():
    # Clear output and display new button
    clear_output(wait=True)
    inject_floating_button()
    # Clone the repository if it doesn't exist
    if not os.path.exists(LOCAL_DIR):
        !git clone {REPO_URL} {LOCAL_DIR}
    else:
        # Pull the latest changes if the repo already exists
        !cd {LOCAL_DIR} && git pull

    # Append the repository to the system path
    if LOCAL_DIR not in sys.path:
        sys.path.append(LOCAL_DIR + "/src")
    print(f"Repository at '{LOCAL_DIR}' is ready for imports.")

# Register the Python function as a callable function for JavaScript
from google.colab import output
output.register_callback('notebook.reload_repo', clone_and_import)

# Inject the button
inject_floating_button()

# Initial setup
clone_and_import()

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 5 (delta 4), reused 5 (delta 4), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 440 bytes | 440.00 KiB/s, done.
From https://github.com/rosinusserrano/autoencoding_experiments
   6bf16f0..47eb11e  main       -> origin/main
Updating 6bf16f0..47eb11e
Fast-forward
 src/logger/caveman.py | 4 ++--
 1 file changed, 2 insertions(+), 2 deletions(-)
Repository at '/content/autoencoding_experiments' is ready for imports.


In [24]:
%reload_ext autoreload
%autoreload 2

In [25]:
import datasets
import logger
import models
import utils

model_config = models.autoencoder.AutoencoderConfig()

dataset_config = datasets.DatasetConfig(
    dataset_name="cifar10",
    batch_size=32,
    validation_split=0.1,
)

train_config = utils.train.TrainConfig(
    optimizer="adam",
    learning_rate=0.001,
    n_epochs=20,
)

logger = logger.caveman.CavemanLogger(root_dir="testrun")

models.autoencoder.run(
    model_config=model_config,
    dataset_config=dataset_config,
    train_config=train_config,
    logger=logger,
    validation_interval=1,
    test_interval=1,
    visualization_interval=1,
)

Files already downloaded and verified
Files already downloaded and verified
On device: cuda
EPOCH 0: train_loss 0.1639
EPOCH 0: val_loss 0.1556
EPOCH 0: test_loss 0.1556
EPOCH 1: train_loss 0.1556
EPOCH 1: val_loss 0.1556
EPOCH 1: test_loss 0.1556
EPOCH 2: train_loss 0.1556
EPOCH 2: val_loss 0.1556
EPOCH 2: test_loss 0.1556
EPOCH 3: train_loss 0.1556
EPOCH 3: val_loss 0.1556
EPOCH 3: test_loss 0.1556
EPOCH 4: train_loss 0.1556
EPOCH 4: val_loss 0.1556
EPOCH 4: test_loss 0.1556
EPOCH 5: train_loss 0.1556
EPOCH 5: val_loss 0.1556
EPOCH 5: test_loss 0.1556
EPOCH 6: train_loss 0.0865
EPOCH 6: val_loss 0.0000
EPOCH 6: test_loss 0.0000
EPOCH 7: train_loss 0.0000
EPOCH 7: val_loss 0.0000
EPOCH 7: test_loss 0.0000
EPOCH 8: train_loss 0.0000
EPOCH 8: val_loss 0.0000
EPOCH 8: test_loss 0.0000
EPOCH 9: train_loss 0.0000
EPOCH 9: val_loss 0.0000
EPOCH 9: test_loss 0.0000
EPOCH 10: train_loss 0.0000
EPOCH 10: val_loss 0.0000
EPOCH 10: test_loss 0.0000


KeyboardInterrupt: 